### **Búsqueda y Minería de Información 2021-22**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Implementación de un motor de búsqueda**

Fechas:

* Comienzo: lunes 7 / martes 8 de febrero
* Entrega: lunes 21 / martes 22 de febrero (14:00)

# Introducción

## Objetivos

Los objetivos de esta práctica son:

* La iniciación a la implementación de un motor de búsqueda.
*	Una primera comprensión de los elementos básicos necesarios para implementar un motor completo.
*	La iniciación al uso de la librería [Whoosh](https://whoosh.readthedocs.io/en/latest/intro.html) en Python para la creación y utilización de índices, funcionalidades de búsqueda en texto.
*	La iniciación a la implementación de una función de ránking sencilla.

Los documentos que se indexarán en esta práctica, y sobre los que se realizarán consultas de búsqueda serán documentos HTML, que deberán ser tratados para extraer y procesar el texto contenido en ellos. 

La práctica plantea como punto de partida una pequeña API general sencilla, que pueda implementarse de diferentes maneras, como así se hará en esta práctica y las siguientes. A modo de toma de contacto y arranque de la asignatura, en esta primera práctica se completará una implementación de la API utilizando Whoosh, con lo que resultará bastante trivial la solución (en cuanto a la cantidad de código a escribir). En la siguiente práctica el estudiante desarrollará sus propias implementaciones, sustituyendo el uso de Whoosh que vamos a hacer en esta primera práctica.

En términos de operaciones propias de un motor de búsqueda, en esta práctica el estudiante se encargará fundamentalmente de:

a) En el proceso de indexación: recorrer los documentos de texto de una colección dada, eliminar del contenido posibles marcas tales como html, y enviar el texto a indexar por parte de Whoosh. 

b) En el proceso de responder consultas: implementar una primera versión sencilla de una o dos funciones de ránking en el modelo vectorial, junto con alguna pequeña estructura auxiliar.

## Material proporcionado

Se proporcionan (bien en el curso de Moodle o dentro de este documento):

*	Varias clases e interfaces Python a lo largo de este *notebook*, desde las que el estudiante partirá para completar código e integrará las suyas propias. 
En particular, la función **main** implementa un programa que deberá funcionar con el código a implementar por el estudiante. Además, se proporciona una celda con código que muestra las funciones más útiles de la API de Whoosh.
*	Una pequeña colección <ins>docs1k.zip</ins> con aproximadamente 1.000 documentos HTML, y un pequeño fichero <ins>urls.txt</ins>. Ambas representan colecciones de prueba para depurar las implementaciones y comprobar su corrección.
*	Un documento de texto <ins>output.txt</ins> con la salida estándar que deberá producir la ejecución de la función main.

In [1]:
import math
from abc import ABC, abstractmethod


"""
    This is an abstract class for the search engines
"""
class Searcher(ABC):
    def __init__(self, index):
        self.index = index
    @abstractmethod
    def search(self, query, cutoff):
        """ Returns a list of documents built as a pair of path and score """

In [4]:
# Whoosh API
import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os, os.path
import shutil

Document = Schema(
        path=ID(stored=True),
        content=TEXT(vector=Format))

def whooshexample_buildindex(dir, urls):
    if os.path.exists(dir): shutil.rmtree(dir)
    os.makedirs(dir)
    writer = whoosh.index.create_in(dir, Document).writer()
    for url in urls:
        writer.add_document(path=url, content=BeautifulSoup(urlopen(url).read(), "lxml").text)
    writer.commit()

def whooshexample_search(dir, query):
    index = whoosh.index.open_dir(dir)
    searcher = index.searcher()
    qparser = QueryParser("content", schema=index.schema)
    print("Search results for '", query, "'")
    for docid, score in searcher.search(qparser.parse(query)).items():
        print(score, "\t", index.reader().stored_fields(docid)['path'])
    print()

def whooshexample_examine(dir, term, docid, n):
    reader = whoosh.index.open_dir(dir).reader()
    print("Total nº of documents in the collection:", reader.doc_count())
    print("Total frequency of '", term, "':", reader.frequency("content", term))
    print("Nº documents containing '", term, "':", reader.doc_frequency("content", term))
    for p in reader.postings("content", term).items_as("frequency") if reader.doc_frequency("content", term) > 0 else []:
        print("\tFrequency of '", term, "' in document", p[0], ":", p[1])
    raw_vec = reader.vector(docid, "content")
    raw_vec.skip_to(term)
    if raw_vec.id() == term:
        print("Frequency of '", raw_vec.id(), "' in document", docid, reader.stored_fields(docid)['path'], ":", raw_vec.value_as("frequency"))
    else:
        print("Term '", term, "' not found in document", docid)
    print("Top", n, "most frequent terms in document", docid, reader.stored_fields(docid)['path']) 
    vec = reader.vector(docid, "content").items_as("frequency")
    for p in sorted(vec, key=lambda x: x[1], reverse=True)[0:n]:
        print("\t", p)
    print()

urls = ["https://en.wikipedia.org/wiki/Simpson's_paradox", 
        "https://en.wikipedia.org/wiki/Bias",
        "https://en.wikipedia.org/wiki/Entropy"]

dir = "index/whoosh/example/urls"

whooshexample_buildindex(dir, urls)
whooshexample_search(dir, "probability")
whooshexample_examine(dir, "probability", 0, 5)

Search results for ' probability '
1.4883011845828815 	 https://en.wikipedia.org/wiki/Simpson's_paradox
1.4264614680779018 	 https://en.wikipedia.org/wiki/Entropy
0.647128995820211 	 https://en.wikipedia.org/wiki/Bias

Total nº of documents in the collection: 3
Total frequency of ' probability ': 29.0
Nº documents containing ' probability ': 3
	Frequency of ' probability ' in document 0 : 12
	Frequency of ' probability ' in document 1 : 1
	Frequency of ' probability ' in document 2 : 16
Frequency of ' probability ' in document 0 https://en.wikipedia.org/wiki/Simpson's_paradox : 12
Top 5 most frequent terms in document 0 https://en.wikipedia.org/wiki/Simpson's_paradox
	 ('simpson', 54)
	 ('mw', 52)
	 ('paradox', 52)
	 ('output', 51)
	 ('parser', 51)



## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado.  

El peso de la nota de esta práctica en la calificación final de prácticas es del **20%**.

La calificación se basará en a) el **número** de ejercicios realizados y b) la **calidad** de los mismos. 
La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan, **sin ninguna modificación**. El profesor comprobará este aspecto añadiendo los módulos entregados por el estudiante a los módulos facilitados en la práctica, ejecutando la función main así como otros main de prueba adicionales.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

En concreto, se debe documentar:

- Qué version(es) del modelo vectorial se ha(n) implementado en el ejercicio 2.
- Cómo se ha conseguido colocar un documento en la primera posición de ránking, para cada buscador implementado en el ejercicio 2.
- El trabajo realizado en el ejercicio 3. 
- Y cualquier otro aspecto que el estudiante considere oportuno destacar.


## Indicaciones

Se podrán definir clases adicionales a las que se indican en el enunciado, por ejemplo, para reutilizar código. Y el estudiante podrá utilizar o no el software que se le proporciona, con la siguiente limitación: 

*	No deberá editarse el software proporcionado más allá de donde se indica explícitamente.
*	El programa **main deberá ejecutar** correctamente.

# Ejercicio 1: Implementación basada en Whoosh

Implementar las clases y módulos necesarios para que el programa main funcione. Se deja al estudiante deducir alguna de las relaciones jerárquicas entre las clases Python.

## Ejercicio 1.1: Indexación (3pt)

Definir las siguientes clases:

* Index.
* WhooshIndex, como subclase de Index.
*	WhooshBuilder, como subclase de Builder.

Se sugiere utilizar dos “fields” en el esquema de documentos de Whoosh: la ruta del documento (dirección Web o ruta en disco), y el contenido del documento.

La entrada para construir el índice (método Builder.build()) podrá ser a) un fichero de texto con direcciones Web (una por línea); b) una carpeta del disco (se indexarán todos los ficheros de la carpeta, sin entrar en subcarpetas); o c) un archivo zip que contiene archivos comprimidos a indexar. Supondremos que el contenido a indexar es siempre HTML.

In [14]:
class Index(ABC):
    def __init__ (self, path):

        self.path = path

    @abstractmethod
    def doc_freq(self, term: str):
        pass

    @abstractmethod
    def ndocs(self):
        pass
        #return self.documents.keys.len()

    @abstractmethod
    def doc_ids(self):
        pass

    @abstractmethod     
    def doc_terms(doc_id):
        pass
    @abstractmethod
    def all_terms():
        pass
        #return self.terms.keys

    @abstractmethod
    def term_freq(word, doc_id):
        pass

    @abstractmethod
    def all_terms_with_freq():
        pass

    @abstractmethod
    def doc_path(doc_id: str):
        pass

    @abstractmethod
    def total_freq(word):
        pass

    @abstractmethod
    def commit():
        pass
            
            

In [16]:
import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser
import zipfile

# A schema in Whoosh is the set of possible fields in a document in
# the search space. We just define a simple 'Document' schema
Document = Schema(
    path=ID(stored=True),
    content=TEXT(vector=Format))


class Builder(ABC):

    def __init__(self, path):
        self.path = path

    @abstractmethod
    def add_document(self, path, content):
        pass

    @abstractmethod
    def commit(self):
        pass

    # @abstractmethod
    # def writer(self):
    #     pass

    def build(self, collection):
        if os.path.isdir(collection):
            content = os.listdir(collection)
            for f in content:
                file = os.path.join(collection, f)
                with open(file) as fp:
                    # print('Se añade el documento '+ fp.read() + 'al indice de ' + self.path)
                    self.writer.add_document(
                        path=file, content=fp.read())
                    fp.close()
                
        elif os.path.isfile(collection):
            if zipfile.is_zipfile(collection):
                zip = zipfile.ZipFile(collection)
                content = zip.namelist()
                # print('ES UN ZIP: ')
                # print(content)

                for f in content:
                    with zip.open(f) as fp:
                        if f[-1] == '/':
                            pass
                        else:
                            self.writer.add_document(
                                path=f, content=BeautifulSoup(fp, "lxml").text)
            elif collection.endswith('.txt'):
                with open(collection) as fp:
                    for url in fp:
                        print('Se añade el documento '+ url + 'al indice de ' + self.path)
                        self.writer.add_document(path=url, content=BeautifulSoup(
                            urlopen(url).read(), "lxml").text)
            else:
                print("Error en los argumentos. Tipo de archivo no soportado")


class WhooshBuilder(Builder, object):
    def __init__(self, path):
        super().__init__(path)
        self.writer = whoosh.index.create_in(self.path, Document).writer()

    def commit(self):
        #return WhooshIndex(self.path).commit()
        return self.writer.commit()

    def add_document(self, dir, content):
        print('Se añade el documento '+ content + 'al indice de ' + dir)
        if whoosh.index.exists_in(self.path) :
            return whoosh.index.open_dir(self.path).writer().add_document(path=dir, content=content)
        else:
            return whoosh.index.create_in(self.path, Document).writer().add_document(path=dir, content=content)
    
    # def writer(self):
    #     return whoosh.index.create_in(self.path, Document).writer()


class WhooshIndex(Index):
    def __init__(self, path):
        super().__init__(path)
        if not os.path.exists(path):
            os.mkdir(path)
        self.index = whoosh.index.open_dir(path)

    def doc_freq(self, term: str):
        return self.index.reader().doc_frequency("content", term)

    def ndocs(self):
        return self.index.reader().doc_count()
    
    def doc_terms(self, docid):
        return self.index.reader().stored_fields(docid)['path']

    def doc_ids(self):
        return self.index.reader().all_doc_ids()

    def all_terms(self):
        return list(self.index.reader().all_terms())
        

    def term_freq(self, word, doc_id):
        for p in list(self.index.reader().postings("content", word).items_as("frequency")):
            if p[0] == doc_id:
                return p[1]
        return 0


    def all_terms_with_freq(self):
        response = list()
        for t in self.all_terms():
            response.append((str(t[1].decode('utf-8')),str(self.doc_freq(t[1].decode('utf-8')))))
        return response

    def doc_path(self, doc_id: str):
        return self.index.reader().stored_fields(doc_id)['path']

    def total_freq(self, word):
        return self.index.reader().frequency("content", word)

    def commit(self):
        return self.index.writer().commit()


### Explicación/documentación

*(por hacer)*

## Ejercicio 1.2: Búsqueda (1.5pt)

Implementar la clase WhooshSearcher como subclase de Searcher.

In [7]:
class WhooshSearcher(Searcher):
    def __init__(self, indexPath):
        self.index = whoosh.index.open_dir(indexPath)

    def search(self, query, cutoff):
        results = list()  
        searcher = self.index.searcher()
        qparser = QueryParser("content", schema=self.index.schema)
        print("Search results for '", query, "'")
        for docid, score in searcher.search(qparser.parse(query)).items():
            results.append([score, self.index.reader().stored_fields(docid)['path']])
            if len(results) >= 5:
                return results
        return results

### Explicación/documentación

*(por hacer)*

# Ejercicio 2: Modelo vectorial

Implementar dos modelos de ránking propios, basados en el modelo vectorial.

## Ejercicio 2.1: Producto escalar (2pt)

Implementar un modelo vectorial propio que utilice el producto escalar (sin dividir por las normas de los vectores) como función de ránking, por medio de la clase VSMDotProductSearcher, como subclase de Searcher.

Este modelo hará uso de la clase Index y se podrá probar con la implementación WhooshIndex (puedes ver un ejemplo de esto en la función main).

Además, la clase VSMDotProductSearcher será intercambiable con WhooshSearcher, como se puede ver en main, donde la función test_search utiliza una implementación u otra sin distinción.

Para simplificar, aplicar a las consultas simplemente una separación de palabras por espacios en blanco y normalización a minúsculas.

In [22]:
    import nltk
    nltk.download('punkt')
    class VSMDotProductSearcher(Searcher):
        def __init__(self, index):
            self.index = index

        def search(self, query, cutoff):
            table = {}
            score = {}
            producto = 0.0
            tokens = nltk.word_tokenize(query)
            print(tokens)
            for word in tokens:
                    if((word != "")and (word in table)):
                       num = table[word]
                       table[word] = num + 1
                    elif(word != ""): 
                        table[word] = 1

            dic = sorted(table.items(), key = lambda asd:asd[1], reverse = True)
            print(dic)
            for doc in self.index.doc_ids():
                for i in range(0, len(dic)):
                    key = dic[i][0]
                    value = dic[i][1]
                    nmatches = self.index.term_freq(key, doc)
                    producto += (nmatches * value)
                    print(key, "in document", self.index.doc_path(doc), ":nmatches = ", nmatches, "* ", value,"= producto = ", producto)
                score[self.index.doc_terms(doc)]= producto
                producto = 0.0 
            results = sorted(score.items(), key = lambda asd:asd[1], reverse = True)
            print("Search results for '", query, "'")
            return results

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Explicación/documentación

*(por hacer)*

### Ejercicio

Añadir a mano un documento a la colección docs1k.zip de manera que aparezca el primero para la consulta “obama family tree” para este buscador. Documentar cómo se ha conseguido y por qué resulta así.

*(por hacer)*

## Ejercicio 2.2: Coseno (1.5pt)

Refinar la implementación del modelo para que calcule el coseno, definiendo para ello una clase VSMCosineSearcher. Para ello se necesitará extender WhooshBuilder con el cálculo de los módulos de los vectores, que deberán almacenarse en un fichero, en la carpeta de índice junto a los ficheros que genera Whoosh. 

Pensad en qué parte del diseño interesa hacer esto, en concreto, qué clase y en qué momento tendría que calcular, devolver y/o almacenar estos módulos.

In [ ]:
    class VSMCosineSearcher(VSMDotProductSearcher):
        ## TODO ##
        # Your code here #
        pass

### Explicación/documentación

*(por hacer)*

### Ejercicio

Añadir a mano un documento a la colección docs1k.zip de manera que aparezca el primero para la consulta “obama family tree” para este buscador. Documentar cómo se ha conseguido y por qué resulta así.

*(por hacer)*

# Ejercicio 3: Estadísticas de frecuencias (2pt)

Utilizando las funcionalidades de la clase Index, implementar una función term_stats que calcule a) las frecuencias totales en la colección de los términos, ordenadas de mayor a menor, y b) el número de documentos que contiene cada término, igualmente de mayor a menor. Visualizar las estadísticas obtenidas en dos gráficas en escala log-log (por cada colección –seis gráficas en total), que se mostrarán en el cuaderno entregado.

In [ ]:
    def term_stats(index):
        ## TODO ##
        # Your code here #
        pass

### Explicación/documentación

*(por hacer)*

# Programa de prueba **main**

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta *collections* en el mismo directorio que este *notebook*. El fichero *toy.zip* hay que descomprimirlo para indexar la carpeta que contiene.

## Función **main**

In [23]:
import os
import shutil
import time


def main():
    index_root_dir = "./index/"
    collections_root_dir = "./collections/"
    test_collection(collections_root_dir + "toy/",
                    index_root_dir + "toy", "cc", "aa dd")
    test_collection(collections_root_dir + "urls.txt", index_root_dir +
                    "urls", "wikipedia", "information probability")
    test_collection(collections_root_dir + "docs1k.zip",
                    index_root_dir + "docs", "seat", "obama family tree")


def clear(index_path: str):
    if os.path.exists(index_path):
        shutil.rmtree(index_path)
    else:
       print("Creating " + index_path)
    os.makedirs(index_path)


def test_collection(collection_path: str, index_path: str, word: str, query: str):
    start_time = time.time()
    print("=================================================================")
    print("Testing indices and search on " + collection_path)

    # Let's create the folder if it did not exist
    # and delete the index if it did
    clear(index_path)

    # We now test building an index
    test_build(WhooshBuilder(index_path), collection_path)

    # We now inspect the index
    index = WhooshIndex(index_path)
    test_read(index, word)

    print("------------------------------")
    print("Checking search results")
    test_search(WhooshSearcher(index_path), query, 5)
    test_search(VSMDotProductSearcher(WhooshIndex(index_path)), query, 5) #TODO
    # test_search(VSMCosineSearcher(WhooshIndex(index_path)), query, 5) #TODO


def test_build(builder, collection):
    stamp = time.time()
    print("Building index with", type(builder))
    print("Collection:", collection)
    # this function should index the recieved collection and add it to the index
    builder.build(collection)
    # when we commit, the information in the index becomes persistent
    # we can also save any extra information we may need
    # (and that cannot be computed until the entire collection is scanned/indexed)
    builder.commit()
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_read(index, word):
    stamp = time.time()
    print("Reading index with", type(index))
    print("Collection size:", index.ndocs())
    print("Vocabulary size:", len(index.all_terms()))
    terms = index.all_terms_with_freq()
    terms.sort(key=lambda tup: tup[1], reverse=True)
    print("  Top 5 most frequent terms:")
    for term in terms[0:5]:
        print("\t" + term[0] + "\t" + str(term[1]) +
              "=" + str(index.total_freq(term)))
    print(terms[:30])
    # more tests
    doc_id = 0
    print()
    print("  Frequency of word \"" + word + "\" in document " + str(doc_id) +
          " - " + index.doc_path(doc_id) + ": " + str(index.term_freq(word, doc_id)))
    print("  Total frequency of word \"" + word + "\" in the collection: " +
          str(index.total_freq(word)) + " occurrences over " + str(index.doc_freq(word)) + " documents")
    print("  Docs containing the word'" + word + "':", index.doc_freq(word))
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_search(engine, query, cutoff):
    stamp = time.time()
    print("  " + engine.__class__.__name__ + " for query '" + query + "'")
    for path, score in engine.search(query, cutoff):
        print(score, "\t", path)
    print()
    print("Done (", time.time() - stamp, "seconds )")
    print()


main()


Testing indices and search on ./collections/toy/
Building index with <class '__main__.WhooshBuilder'>
Collection: ./collections/toy/
Done ( 0.0074045658111572266 seconds )

Reading index with <class '__main__.WhooshIndex'>
Collection size: 4
Vocabulary size: 43
  Top 5 most frequent terms:
	aa	2=9.0
	bb	2=5.0
	cc	2=3.0
	ache	1=1.0
	aye	1=1.0
[('aa', '2'), ('bb', '2'), ('cc', '2'), ('ache', '1'), ('aye', '1'), ('coil', '1'), ('come', '1'), ('consummation', '1'), ('dd', '1'), ('death', '1'), ('devoutly', '1'), ('die', '1'), ('dream', '1'), ('dreams', '1'), ('end', '1'), ('flesh', '1'), ('give', '1'), ('heart', '1'), ('heir', '1'), ('more', '1'), ('mortal', '1'), ('must', '1'), ('natural', '1'), ('no', '1'), ('off', '1'), ('opposing', '1'), ('pause', '1'), ('perchance', '1'), ('rub', '1'), ('say', '1')]

  Frequency of word "cc" in document 0 - ./collections/toy/d3.txt: 1
  Total frequency of word "cc" in the collection: 3.0 occurrences over 2 documents
  Docs containing the word'cc': 2
D

KeyboardInterrupt: ignored

### Salida obtenida por el estudiante

*(por hacer)*